# Interfax

this is very old code

In [9]:
import requests
import numpy as np
import datetime
import re
from bs4 import BeautifulSoup

from fake_useragent import UserAgent

import time
import datetime
import pandas as pd
import pickle
import time

from tqdm import tqdm
from joblib import Parallel, delayed

In [6]:
def monthlength(month,year):
    if year % 4 == 0:
         VisYear = 29
    else:
         VisYear = 28
    return [31,VisYear,31,30,31,30,31,31,30,31,30,31][month]

def days_of_year(year):
    dates = [ ]
    for mon in range(12):
        cur_month = [(year, mon+1, day) for day in range(1, monthlength(mon, year)+1)]
        dates.extend(cur_month)
    return dates 

def html_stripper(text):
    return re.sub('<[^<]+?>', '', str(text)) 

In [7]:
from multiprocessing.pool import ThreadPool

def Separator(vect, part):
    n = len(vect)
    vec_parts = [round(n/part)*i for i in range(part)]
    vec_parts.append(n)
    out = [vect[vec_parts[i]:vec_parts[i+1]] for i in range(part)]
    return(out)

def Map(vect, parser_function):
    out = [parser_function(item) for item in vect]
    return(out)

def Reduce(l):
    ll = [ ]
    for item in l:
        ll.extend(item)
    return(ll)

def MRDownloader(what, parts, parser_function):
    # Map - шаг 
    separatorlist = Separator(what, parts)
    def Mp(what):
        return Map(what, parser_function=parser_function)
    # Скачиваем 
    pool = ThreadPool(parts)
    l = pool.map(Mp, separatorlist)
    # Reduce - шаг
    itog = Reduce(l)
    return(itog)    

# 1. Download hrefs

In [8]:
def news_data_get(year,month,day,rubrics='all'):
    if len(str(day)) < 2:
        day = '0' + str(day)
        
    if len(str(month)) < 2:
        month = '0' + str(month)
        
    mainpage = 'http://www.interfax.ru/news/{}/{}/{}/{}/'.format(year, month, day, rubrics)
    response = requests.get(mainpage)
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    
    i_page = int(soup.find('div', {'class' : 'pages'}).text[-2])
    
    all_news = [ ]
    for i in range(1, i_page+1):
        response = requests.get(mainpage + 'page_' + str(i))
        html = response.content
        soup = BeautifulSoup(html, 'html.parser')
        cur_news = soup.find('div', {'class' : 'an'}).find_all('div')        
        all_news.extend([nw for nw in cur_news  if len("" if not nw.get('data-id') else nw.get('data-id')) > 0])
    return all_news

def news_to_df(news):
    new = { }
    new['date'] = news.get('data-id')
    new['time'] = news.find('span').text
    new['title'] = news.find('a').text
    new['href'] = news.find('a').get('href')
    return new

In [ ]:
# Cбор данных с нуля
# years = list(range(2019,2021))   # цикл пойдёт по годам и по рубрикам 

# itog_titles = [ ]
# for yr in tqdm_notebook(years, desc='Years'):
#     days = days_of_year(yr)
#     if yr == 2008:
#         days = days[41:]
#     for date in tqdm_notebook(days, desc='Days'):
#         all_news = news_data_get(date[0],date[1],date[2])
#         for new in all_news:
#             cur_df = news_to_df(new)
#             cur_df['year'] = date[0] 
#             cur_df['month'] = date[1]
#             cur_df['day'] = date[2]
#             itog_titles.append(cur_df)

In [10]:
# Досбор
yr = 2020 
days = days_of_year(2020)[152:300]

itog_titles = [ ]
for date in tqdm(days):
    all_news = news_data_get(date[0],date[1],date[2])
    for new in all_news:
        cur_df = news_to_df(new)
        cur_df['year'] = date[0] 
        cur_df['month'] = date[1]
        cur_df['day'] = date[2]
        itog_titles.append(cur_df)

100%|██████████| 148/148 [25:55<00:00, 10.51s/it]


In [11]:
len(itog_titles)

20384

In [14]:
!ls news_data/3.interfax

In [15]:
with open('news_data/3.interfax/titles_new.pickle', 'wb') as f:
    pickle.dump(itog_titles, f)

In [16]:
df = pd.DataFrame(itog_titles)
print(df.shape)
df.head( )

(20384, 7)


,date,time,title,href,year,month,day
0,711320,23:50,В Подмосковье с 3 июня возобновят оплату транс...,/russia/711320,2020,6,1
1,711319,23:33,Ракова сообщила о проведении 85 тыс. тестов на...,/moscow/711319,2020,6,1
2,711318,23:10,Временные коронавирусные госпитали в Москве ос...,/moscow/711318,2020,6,1
3,711317,23:04,Бывший топ-менеджер Litasco обнаружился в схем...,/business/711317,2020,6,1
4,711316,22:58,В ООН призвали США расследовать действия полиц...,/world/711316,2020,6,1


# 2. Download news

In [17]:
def get_interfax_news(item_from_vect):
    url = 'http://www.interfax.ru' + item_from_vect['href']
    try:
        response = requests.get(url)
        html = response.content
        soup = BeautifulSoup(html,'html.parser')
        txt = soup.find("article", {"itemprop": "articleBody"}).text
        item_from_vect['text'] = txt
        return item_from_vect
    except:
        print(url)
        return { }
    
#get_interfax_news({'href' : '/russia/97031'})
get_interfax_news(itog_titles[0])

{'date': '711320',
 'time': '23:50',
 'title': 'В Подмосковье с 3 июня возобновят оплату транспорта банковскими картами',
 'href': '/russia/711320',
 'year': 2020,
 'month': 6,
 'day': 1,
 'text': '\nВ Подмосковье с 3 июня возобновят оплату транспорта банковскими картами\nМосква. 1 июня. INTERFAX.RU - С 3 июня возобновляется возможность оплаты проезда в общественном транспорте Подмосковья при помощи банковской карты, сообщает региональный штаб по по контролю и мониторингу ситуации с коронавирусом.\n"Андрей Воробьёв подписал постановление, ослабляющее в Подмосковье действие ограничений, введённых на период пандемии коронавируса. Со среды 3 июня в регионе (...) разрешается оплата банковскими картами при проезде на транспорте", - говорится в сообщении.\nРанее сообщалось, что в понедельник губернатор Московской области Андрей Воробьев подписал постановление, согласно которому с 3 июня могут возобновить работу религиозные организации.\nТакже снимаются ограничения с работы салонов красоты, С

In [18]:
%%time
itog_news = MRDownloader(itog_titles, 5, parser_function=get_interfax_news)
len(itog_news)

http://www.interfax.ruhttps://www.sport-interfax.ru/728969
http://www.interfax.ruhttps://www.sport-interfax.ru/723768
http://www.interfax.ruhttps://www.sport-interfax.ru/723767
http://www.interfax.ruhttps://www.sport-interfax.ru/719677
http://www.interfax.ruhttps://www.sport-interfax.ru/711306
http://www.interfax.ruhttps://www.sport-interfax.ru/723760
http://www.interfax.ruhttps://www.sport-interfax.ru/711303
http://www.interfax.ruhttps://www.sport-interfax.ru/728948
http://www.interfax.ruhttps://www.sport-interfax.ru/719666
http://www.interfax.ruhttps://www.sport-interfax.ru/715272
http://www.interfax.ruhttps://www.sport-interfax.ru/728943
http://www.interfax.ruhttps://www.sport-interfax.ru/719660
http://www.interfax.ruhttps://www.sport-interfax.ru/728941
http://www.interfax.ruhttps://www.sport-interfax.ru/723744
http://www.interfax.ruhttps://www.sport-interfax.ru/723742
http://www.interfax.ruhttps://www.sport-interfax.ru/719651
http://www.interfax.ruhttps://www.sport-interfax.ru/7112

http://www.interfax.ruhttps://www.sport-interfax.ru/729273
http://www.interfax.ruhttps://www.sport-interfax.ru/719971
http://www.interfax.ruhttps://www.sport-interfax.ru/711632
http://www.interfax.ruhttps://www.sport-interfax.ru/724338
http://www.interfax.ruhttps://www.sport-interfax.ru/724336
http://www.interfax.ruhttps://www.sport-interfax.ru/711624
http://www.interfax.ruhttps://www.sport-interfax.ru/719973
http://www.interfax.ruhttps://www.sport-interfax.ru/715792
http://www.interfax.ruhttps://www.sport-interfax.ru/715782
http://www.interfax.ruhttps://www.sport-interfax.ru/711599
http://www.interfax.ruhttps://www.sport-interfax.ru/720150
http://www.interfax.ruhttps://www.sport-interfax.ru/720145
http://www.interfax.ruhttps://www.sport-interfax.ru/729233
http://www.interfax.ruhttps://www.sport-interfax.ru/720135
http://www.interfax.ruhttps://www.sport-interfax.ru/729602
http://www.interfax.ruhttps://www.sport-interfax.ru/724296
http://www.interfax.ruhttps://www.sport-interfax.ru/7242

http://www.interfax.ruhttps://www.sport-interfax.ru/711925
http://www.interfax.ruhttps://www.sport-interfax.ru/729911
http://www.interfax.ruhttps://www.sport-interfax.ru/716251
http://www.interfax.ruhttps://www.sport-interfax.ru/716246
http://www.interfax.ruhttps://www.sport-interfax.ru/729898
http://www.interfax.ruhttps://www.sport-interfax.ru/724546
http://www.interfax.ruhttps://www.sport-interfax.ru/729890
http://www.interfax.ruhttps://www.sport-interfax.ru/720665
http://www.interfax.ruhttps://www.sport-interfax.ru/720659
http://www.interfax.ruhttps://www.sport-interfax.ru/720657
http://www.interfax.ruhttps://www.sport-interfax.ru/712117
http://www.interfax.ruhttps://www.sport-interfax.ru/716210
http://www.interfax.ruhttps://www.sport-interfax.ru/720633
http://www.interfax.ruhttps://www.sport-interfax.ru/724707
http://www.interfax.ruhttps://www.sport-interfax.ru/720625
http://www.interfax.ruhttps://www.sport-interfax.ru/716178
http://www.interfax.ruhttps://www.sport-interfax.ru/7165

http://www.interfax.ruhttps://www.sport-interfax.ru/725267
http://www.interfax.ruhttps://www.sport-interfax.ru/712711
http://www.interfax.ruhttps://www.sport-interfax.ru/720932
http://www.interfax.ruhttps://www.sport-interfax.ru/720931
http://www.interfax.ruhttps://www.sport-interfax.ru/716870
http://www.interfax.ruhttps://www.sport-interfax.ru/716867
http://www.interfax.ruhttps://www.sport-interfax.ru/712703
http://www.interfax.ruhttps://www.sport-interfax.ru/725255
http://www.interfax.ruhttps://www.sport-interfax.ru/716864
http://www.interfax.ruhttps://www.sport-interfax.ru/720922
http://www.interfax.ruhttps://www.sport-interfax.ru/720921
http://www.interfax.ruhttps://www.sport-interfax.ru/721188
http://www.interfax.ruhttps://www.sport-interfax.ru/730378
http://www.interfax.ruhttps://www.sport-interfax.ru/721181
http://www.interfax.ruhttps://www.sport-interfax.ru/721178
http://www.interfax.ruhttps://www.sport-interfax.ru/712695
http://www.interfax.ruhttps://www.sport-interfax.ru/7168

http://www.interfax.ruhttps://www.sport-interfax.ru/721439
http://www.interfax.ruhttps://www.sport-interfax.ru/730914
http://www.interfax.ruhttps://www.sport-interfax.ru/726618
http://www.interfax.ruhttps://www.sport-interfax.ru/717317
http://www.interfax.ruhttps://www.sport-interfax.ru/726610
http://www.interfax.ruhttps://www.sport-interfax.ru/726609
http://www.interfax.ruhttps://www.sport-interfax.ru/726608
http://www.interfax.ruhttps://www.sport-interfax.ru/726606
http://www.interfax.ruhttps://www.sport-interfax.ru/713086
http://www.interfax.ruhttps://www.sport-interfax.ru/713084
http://www.interfax.ruhttps://www.sport-interfax.ru/721421
http://www.interfax.ruhttps://www.sport-interfax.ru/713077
http://www.interfax.ruhttps://www.sport-interfax.ru/721419
http://www.interfax.ruhttps://www.sport-interfax.ru/721417
http://www.interfax.ruhttps://www.sport-interfax.ru/717297
http://www.interfax.ruhttps://www.sport-interfax.ru/721408
http://www.interfax.ruhttps://www.sport-interfax.ru/7267

http://www.interfax.ruhttps://www.sport-interfax.ru/731488
http://www.interfax.ruhttps://www.sport-interfax.ru/713547
http://www.interfax.ruhttps://www.sport-interfax.ru/717922
http://www.interfax.ruhttps://www.sport-interfax.ru/727240
http://www.interfax.ruhttps://www.sport-interfax.ru/727234
http://www.interfax.ruhttps://www.sport-interfax.ru/713530
http://www.interfax.ruhttps://www.sport-interfax.ru/721945
http://www.interfax.ruhttps://www.sport-interfax.ru/721942
http://www.interfax.ruhttps://www.sport-interfax.ru/727188
http://www.interfax.ruhttps://www.sport-interfax.ru/717885
http://www.interfax.ruhttps://www.sport-interfax.ru/717877
http://www.interfax.ruhttps://www.sport-interfax.ru/722113
http://www.interfax.ruhttps://www.sport-interfax.ru/722112
http://www.interfax.ruhttps://www.sport-interfax.ru/713467
http://www.interfax.ruhttps://www.sport-interfax.ru/717854
http://www.interfax.ruhttps://www.sport-interfax.ru/717852
http://www.interfax.ruhttps://www.sport-interfax.ru/7178

http://www.interfax.ruhttps://www.sport-interfax.ru/731925
http://www.interfax.ruhttps://www.sport-interfax.ru/714050
http://www.interfax.ruhttps://www.sport-interfax.ru/718382
http://www.interfax.ruhttps://www.sport-interfax.ru/718373
http://www.interfax.ruhttps://www.sport-interfax.ru/718366
http://www.interfax.ruhttps://www.sport-interfax.ru/714026
http://www.interfax.ruhttps://www.sport-interfax.ru/718352
http://www.interfax.ruhttps://www.sport-interfax.ru/714020
http://www.interfax.ruhttps://www.sport-interfax.ru/722629
http://www.interfax.ruhttps://www.sport-interfax.ru/722624
http://www.interfax.ruhttps://www.sport-interfax.ru/727546
http://www.interfax.ruhttps://www.sport-interfax.ru/718335
http://www.interfax.ruhttps://www.sport-interfax.ru/714003
http://www.interfax.ruhttps://www.sport-interfax.ru/732025
http://www.interfax.ruhttps://www.sport-interfax.ru/714001
http://www.interfax.ruhttps://www.sport-interfax.ru/732022
http://www.interfax.ruhttps://www.sport-interfax.ru/7320

http://www.interfax.ruhttps://www.sport-interfax.ru/718865
http://www.interfax.ruhttps://www.sport-interfax.ru/723065
http://www.interfax.ruhttps://www.sport-interfax.ru/718862
http://www.interfax.ruhttps://www.sport-interfax.ru/723060
http://www.interfax.ruhttps://www.sport-interfax.ru/723050http://www.interfax.ruhttps://www.sport-interfax.ru/728347

http://www.interfax.ruhttps://www.sport-interfax.ru/723047
http://www.interfax.ruhttps://www.sport-interfax.ru/718849
http://www.interfax.ruhttps://www.sport-interfax.ru/718832
http://www.interfax.ruhttps://www.sport-interfax.ru/714700
http://www.interfax.ruhttps://www.sport-interfax.ru/718830
http://www.interfax.ruhttps://www.sport-interfax.ru/728322
http://www.interfax.ruhttps://www.sport-interfax.ru/723018
http://www.interfax.ruhttps://www.sport-interfax.ru/728314
http://www.interfax.ruhttps://www.sport-interfax.ru/728303
http://www.interfax.ruhttps://www.sport-interfax.ru/732418
http://www.interfax.ruhttps://www.sport-interfax.ru/7146

http://www.interfax.ruhttps://www.sport-interfax.ru/723502http://www.interfax.ruhttps://www.sport-interfax.ru/715013

http://www.interfax.ruhttps://www.sport-interfax.ru/734008
http://www.interfax.ruhttps://www.sport-interfax.ru/723497
http://www.interfax.ruhttps://www.sport-interfax.ru/734001
http://www.interfax.ruhttps://www.sport-interfax.ru/719277
http://www.interfax.ruhttps://www.sport-interfax.ru/719275
http://www.interfax.ruhttps://www.sport-interfax.ru/734113
http://www.interfax.ruhttps://www.sport-interfax.ru/734111
http://www.interfax.ruhttps://www.sport-interfax.ru/715215
http://www.interfax.ruhttps://www.sport-interfax.ru/715211
http://www.interfax.ruhttps://www.sport-interfax.ru/715201
http://www.interfax.ruhttps://www.sport-interfax.ru/715200
http://www.interfax.ruhttps://www.sport-interfax.ru/723454
http://www.interfax.ruhttps://www.sport-interfax.ru/715194
http://www.interfax.ruhttps://www.sport-interfax.ru/723451
http://www.interfax.ruhttps://www.sport-interfax.ru/7151

20384

In [19]:
itog_news = [itog for itog in itog_news if len(itog.keys()) != 0 ]
len(itog_news)

19200

In [20]:
itog_news[0]

{'date': '711320',
 'time': '23:50',
 'title': 'В Подмосковье с 3 июня возобновят оплату транспорта банковскими картами',
 'href': '/russia/711320',
 'year': 2020,
 'month': 6,
 'day': 1,
 'text': '\nВ Подмосковье с 3 июня возобновят оплату транспорта банковскими картами\nМосква. 1 июня. INTERFAX.RU - С 3 июня возобновляется возможность оплаты проезда в общественном транспорте Подмосковья при помощи банковской карты, сообщает региональный штаб по по контролю и мониторингу ситуации с коронавирусом.\n"Андрей Воробьёв подписал постановление, ослабляющее в Подмосковье действие ограничений, введённых на период пандемии коронавируса. Со среды 3 июня в регионе (...) разрешается оплата банковскими картами при проезде на транспорте", - говорится в сообщении.\nРанее сообщалось, что в понедельник губернатор Московской области Андрей Воробьев подписал постановление, согласно которому с 3 июня могут возобновить работу религиозные организации.\nТакже снимаются ограничения с работы салонов красоты, С

In [21]:
m = [tt['href'] for tt in itog_news]
len(m)

19200

In [22]:
len(set(m))

19155

In [23]:
with open('news_data/interfax_new.pickle', 'wb') as f:
    pickle.dump(itog_news, f)